In [1]:
%pip install pandas
%pip install torch torchvision --index-url https://download.pytorch.org/whl/cu121
import pandas as pd
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

#from PIL import Image (Helps add image processing if needed)
#from torch.utils.data import Dataset (for custom datasets if needed)

Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Preparing the data

# Number of classes in the MNIST dataset (digits 0-9).
num_classes = 10
# (channels, [dimensions of image in pixels]). Greyscale = 1, RGB = 3.
input_shape = (1, 28, 28)

# Instances of Pytorch dataset objects which load and split the MNIST data into train and test sets and transforms the images to tensors.
train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transforms.Compose([transforms.ToTensor()]))
test_dataset = datasets.MNIST(root='./data', train=False, transform=transforms.Compose([transforms.ToTensor()]))
# Objects to provide access to the tensor data during training and testing.
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [3]:
# Building the model (Creating custom CNN 'Net' using nn.Module class)
class Net(nn.Module):
    # Initialization function
    def __init__(self):
        # Constructor which initializes the layers and defines parameters.
        super(Net, self).__init__()
        # convolutional layer for extracting local features/patterns. (input channels, output channels, kernel size, stride).
        self.conv1 = nn.Conv2d(1, 4, 3, 1)
        # max pooling layer for downsampling to retain important features & reduce spatial dimensions. (kernel size, stride).
        self.pool = nn.MaxPool2d(2, 2)
        # dropout layer for regularization. Prevents overfitting. (Probability of element dropped during dropout).
        self.dropout = nn.Dropout2d(0.5)
        # fully connected layer for classification. (input size, output size).
        self.fc1 = nn.Linear(4 * 13 * 13, num_classes)

    # Function to define how the input flows through the layers.
    def forward(self, x):
        # Applies convolutional layer, activation function (ReLU) for non-linearity, and pooling layer to the input.
        x = self.pool(F.relu(self.conv1(x)))
        # Reshapes the output tensor to match the input size of the fully connected layer.
        x = x.view(-1, 4 * 13 * 13)
        # Applies dropout layer to prevent overfitting by randomly zeroing some elements.
        x = self.dropout(x)
        # Flattens and passes the input through the fully connected layer, performs linear transformation for classification.
        x = self.fc1(x)
        # Logarithmic softmax function applied to produce the output probabilities.
        return F.log_softmax(x, dim=1)

model = Net()

In [7]:
# Training the model

# Device selection
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Optimizer using Adam op algo to optimize parameters during training with the set learning rate.
optimizer = optim.Adam(model.parameters(), lr=0.001)
# Loss function (combines softmax activation and negative log-likelihood loss).
criterion = nn.CrossEntropyLoss()

# Performs the training loop for a number of epochs.
def train(model, device, train_loader, optimizer, epoch):
    # Sets model in training mode, so it can update it's parameters during training.
    model.train()
    # Iterates over the loader.
    for batch_idx, (data, target) in enumerate(train_loader):
        # Input data and target labels moved to the GPU/CPU.
        data, target = data.to(device), target.to(device)
        # Optimizer's gradients reset to clear accumulated gradients from pervious batches.
        optimizer.zero_grad()
        # forward pass is performed, obtaining predicted outputs.
        output = model(data)
        # Loss calculated by comparing predicted outputs with target labels.
        loss = criterion(output, target)
        # Gradients are computed and backproped.
        loss.backward()
        # Optimizer updates models parameters.
        optimizer.step()

In [8]:
torch.set_num_threads(2)

# Evaluating the trained model
def test(model, device, test_loader):
    # Sets model in evaluation mode (disables dropout and batch normalization).
    model.eval()
    # Initialization of cumulative loss and correctly predicted samples.
    test_loss = 0
    correct = 0
    # Ensures no gradient calcs are performed during the evaluation, reducing memory consumption.
    with torch.no_grad():
        # Iterates over (image, label) batches in test dataset.
        for data, target in test_loader:
            # Input data and target labels moved to the GPU/CPU.
            data, target = data.to(device), target.to(device)
            # forward pass is performed, obtaining predicted outputs.
            output = model(data)
            # Accumulated test loss between predicted output and target labels.
            test_loss += criterion(output, target).item()
            # Tensor containing the predicted labels by taking the idx of the max value .
            pred = output.argmax(dim=1, keepdim=True)
            # Accumulated correct predictions by comparing predicted labels with target labels.
            correct += pred.eq(target.view_as(pred)).sum().item()
    # Obtains average loss.
    test_loss /= len(test_loader.dataset)
    return test_loss, correct

# Runs training process for n-1 epochs.
for epoch in range(1, 6):
    train(model, device, train_loader, optimizer, epoch)


# Calls the test function after all the epochs and then prints the result
test_loss, correct = test(model, device, test_loader)
print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))


Test set: Average loss: 0.0037, Accuracy: 9655/10000 (96.55%)

